# LHC publications with HEPData records

This notebook counts all Inspire records that have an associated HEPData record using the Inspire API.

Count publications per LHC experiment per year and see which fraction has a HEPData record.

Written by Graeme Watt (Project Manager for HEPData) on 11th October 2017.

## Get results

Define a function to compose an Inspire query.

In [ ]:
def compose_query(collaboration='', year=0, has_hepdata=False):
    """
    Compose an Inspire query for published papers, with optional arguments
    to restrict by collaboration, year, and presence of a HEPData record.
    """

    # Restrict search to papers published (in a refereed journal) that are not conference
    # papers and that appear in the hep-ex or nucl-ex primary arXiv categories.
    query = 'tc p and not tc c and (primarch hep-ex or primarch nucl-ex)'
    
    # Restrict search to a particular experimental collaboration.
    # Empty collaboration searches for papers from all four main LHC collaborations.
    if collaboration:
        query += ' and collaboration:{}'.format(collaboration)
    else:
        query += ' and (collaboration:ALICE or collaboration:ATLAS or ' + \
        'collaboration:CMS or collaboration:LHCb)'
        
    # Restrict search to a particular year.
    if year:
        query += ' and date {}'.format(year)
    
    # Restrict search to publications that have a corresponding HEPData record.
    if has_hepdata:
        query += ' and 035__:HEPDATA'
    
    return query

Define a function to get the number of Inspire records for a given query.

In [ ]:
import requests
import re

def get_num_results(query):
    """
    Make a search using the Inspire API (https://inspirehep.net/info/hep/api)
    with output format MARCXML and returning only the ID number of the first record,
    to save unnecessary output.  Returns the number of records in the search result.
    """
    
    url = 'https://inspirehep.net/search?p=' + query + '&of=xm&rg=1&ot=001'
    
    response = requests.get(url)
    
    m = re.search('Search-Engine-Total-Number-Of-Results: (\d+)', response.text)
    num_results = int(m.group(1)) if m else 0
    
    return num_results

Produce results for all years, then 2010 up to 2018, and the four main LHC collaborations.

In [ ]:
years = [0]
[years.append(year) for year in range(2010, 2019)]
collaborations = ['', 'ALICE', 'ATLAS', 'CMS', 'LHCb']

Loop over all years and collaborations and get number of results from Inspire.

In [ ]:
num_results = {}
num_results_with_hepdata = {}
fraction_of_results_with_hepdata = {}

for year in years:

    num_results[year] = {} 
    num_results_with_hepdata[year] = {}
    fraction_of_results_with_hepdata[year] = {}
    
    for collaboration in collaborations:
        
        query = compose_query(collaboration, year)
        num_results[year][collaboration] = get_num_results(query)
        
        query_with_hepdata = compose_query(collaboration, year, True)
        num_results_with_hepdata[year][collaboration] = get_num_results(query_with_hepdata)
        
        if num_results[year][collaboration] > 0:
            fraction_of_results_with_hepdata[year][collaboration] = \
            float(num_results_with_hepdata[year][collaboration]) / \
            float(num_results[year][collaboration])
        else:
            fraction_of_results_with_hepdata[year][collaboration] = 0
        
        print('Year {}, {} collaboration, {} papers, {} with HEPData, fraction {:.2f}.'.format(
            year if year else 'all',collaboration if collaboration else 'ALICE+ATLAS+CMS+LHCb',
            num_results[year][collaboration],
            num_results_with_hepdata[year][collaboration],
            fraction_of_results_with_hepdata[year][collaboration]))

    print('')

## Make some plots

In [ ]:
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

Take all years together.  Make a bar chart of number of papers for {ALICE, ATLAS, CMS, LHCb} and the subset with a HEPData record.

In [ ]:
labels = [collaboration for collaboration in collaborations if collaboration]
values = [num_results[0][collaboration] for collaboration in collaborations if collaboration]
values_with_hepdata = [num_results_with_hepdata[0][collaboration] 
                       for collaboration in collaborations if collaboration]

plt.figure()
x = 2*labels
y = values + values_with_hepdata
hue = len(labels)*['All papers'] + len(labels)*['With HEPData']
sns.barplot(x, y, hue)
plt.title('LHC publications with HEPData records ({})'.format(date.today()))
plt.xlabel('Collaboration')
plt.ylabel('Number of papers')
plt.savefig('allyears_{}.png'.format(date.today()))
plt.show()

Plot the fraction of papers with a HEPData record for {ALICE, ATLAS, CMS, LHCb}.

In [ ]:
fraction_with_hepdata = [fraction_of_results_with_hepdata[0][collaboration]
                         for collaboration in collaborations if collaboration]
plt.figure()
x = labels
y = fraction_with_hepdata
sns.barplot(x, y)
plt.title('LHC publications with HEPData records ({})'.format(date.today()))
plt.xlabel('Collaboration')
plt.ylabel('Fraction of papers with HEPData')
plt.savefig('allyears_fraction_{}.png'.format(date.today()))
plt.show()

For each of the four experiments {ALICE, ATLAS, CMS, LHCb}, make a bar chart of number of papers versus year.

In [ ]:
plt.figure(figsize=(15,15))
for sub in range(1, 5):
    plt.subplot(2, 2, sub)
    collaboration = collaborations[sub]
    labels = [year for year in years if year]
    values = [num_results[year][collaboration] for year in years if year]
    values_with_hepdata = [num_results_with_hepdata[year][collaboration] 
                           for year in years if year]
    x = 2*labels
    y = values + values_with_hepdata
    hue = len(labels)*['All papers'] + len(labels)*['With HEPData']
    sns.barplot(x, y, hue)
    plt.title(collaboration)
    plt.xlabel('Year')
    plt.ylabel('Number of papers')
plt.suptitle('LHC publications with HEPData records ({})'.format(date.today()), 
             fontsize='xx-large')
plt.savefig('breakdown_{}.png'.format(date.today()))
plt.show()

For each of the four experiments {ALICE, ATLAS, CMS, LHCb}, make a bar chart of the fraction of papers with a HEPData record versus year.

In [ ]:
plt.figure(figsize=(15,15))
for sub in range(1, 5):
    plt.subplot(2, 2, sub)
    collaboration = collaborations[sub]
    labels = [year for year in years if year]
    values = [num_results[year][collaboration] for year in years if year]
    fraction_with_hepdata = [fraction_of_results_with_hepdata[year][collaboration]
                             for year in years if year]
    x = labels
    y = fraction_with_hepdata
    sns.barplot(x, y)
    plt.title(collaboration)
    plt.xlabel('Year')
    plt.ylabel('Fraction of papers with HEPData')
plt.suptitle('LHC publications with HEPData records ({})'.format(date.today()), 
             fontsize='xx-large')
plt.savefig('breakdown_fraction_{}.png'.format(date.today()))
plt.show()

# Get results for your favourite experiment